### Algorithm: CNN-1D with normalization vo
### Dataset: Cricket
### Authors: Edgar Acuna  
### Created: April 4, 2025

In [34]:
import tensorflow

In [35]:
# cnn lstm model
from numpy import mean
from numpy import std, unique
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot

from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, roc_auc_score, ConfusionMatrixDisplay
import tensorflow.keras.layers as layers
import keras
#from keras.layers import TimeDistributed
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv1D, MaxPooling1D, LSTM, TimeDistributed
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from keras.utils import to_categorical

In [36]:
import pandas as pd
from aeon.datasets import load_classification
from sklearn import preprocessing
from matplotlib import pyplot

def load_data(dataset, split="Train"):
    if split=="Train":
        X, y = load_classification(dataset, split="Train")
    else:
        X, y = load_classification(dataset, split="Test")
    print(" Shape of X = ", X.shape)
    print(" Shape of y = ", y.shape)
    #Swap dimensions so features are the third dimension
    X = X.swapaxes(1, 2)
    print(" New shape of X = ", X.shape)
    #prepare y
    y = pd.DataFrame(y)
    enc = preprocessing.OneHotEncoder(dtype=int)
    enc.fit(y)
    y = enc.transform(y).toarray()
    print(" New shape of y = ", y.shape)
    return X, y


In [37]:
# loading Training data
X, y = load_data("Cricket",split="Train")

 Shape of X =  (108, 6, 1197)
 Shape of y =  (108,)
 New shape of X =  (108, 1197, 6)
 New shape of y =  (108, 12)


In [38]:
import numpy as np
yo=np.argmax(y,axis=1)
yo=np.array(yo,dtype=int)
print(yo)

[ 0  0  0  0  0  0  0  0  0  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5
  5  5  5  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  8  8  8
  8  8  8  8  8  8  9  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10 10 10
 11 11 11 11 11 11 11 11 11  1  1  1  1  1  1  1  1  1  2  2  2  2  2  2
  2  2  2  3  3  3  3  3  3  3  3  3]


In [39]:
# loading Testing data
Xt, yt = load_data("Cricket",split="Test")

 Shape of X =  (72, 6, 1197)
 Shape of y =  (72,)
 New shape of X =  (72, 1197, 6)
 New shape of y =  (72, 12)


In [40]:
yto=np.argmax(yt,axis=1)
yto=np.array(yto,dtype=int)
#print(yto)

In [41]:
#class size in testing
import numpy as np
np.sum(yt,axis=0)

array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6])

In [42]:
Xtb=Xt

In [43]:
import numpy as np
Xb= np.concatenate((X, Xt),axis=0)
print(Xb.shape)

(180, 1197, 6)


In [44]:
from sklearn.preprocessing import StandardScaler#
scaler = StandardScaler()
#tempo=X.reshape(X.shape[2],-1).T
#print(tempo.shape)
#X = scaler.fit_transform(X.reshape(X.shape[2], -1)).reshape(X.shape)
Xb = scaler.fit_transform(Xb.reshape(Xb.shape[2], -1)).reshape(Xb.shape)
#Xt = scaler.fit_transform(Xt.reshape(Xt.shape[2], -1)).reshape(Xt.shape)
#X1 = scaler.fit_transform(tempo)
#print(X)
#tempodf=pd.DataFrame(X1)
#print(tempodf.shape)
#pyplot.boxplot(tempodf)
#pyplot.show()

In [45]:
Xn1=Xb[0:108]
print(Xn1.shape)
Xn2=Xb[108:180]
print(Xn2.shape)

(108, 1197, 6)
(72, 1197, 6)


In [46]:
import time
#trainX, trainy, testX, testy = trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.2, random_state=42)
#print(trainX.shape)
#print("trainy",trainy.shape)
trainX=Xn1
trainy=y
testX=Xn2
testy=yt
verbose, epochs, batch_size = 0, 300, 32
n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
print(n_timesteps,n_features,n_outputs)
model = Sequential()
model.add(keras.Input(shape=(n_timesteps,n_features)))
model.add(Conv1D(16, 2, activation="relu"))
model.add(MaxPooling1D(pool_size=2))
#model.add(Dropout(0.5))
model.add(Conv1D(32, 2, activation="relu"))
model.add(MaxPooling1D(pool_size=2))
#model.add(Dropout(0.5))
model.add(Conv1D(64, 2, activation="relu"))
model.add(MaxPooling1D(pool_size=2))
#model.add(Dropout(0.5))
#model.add(Conv1D(128, 3, activation="relu", input_shape=(n_timesteps,n_features)))
#model.add(MaxPooling1D(pool_size=3))
#model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(12, activation = 'softmax'))
start_time = time.time()
model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
model.summary()
model.fit(trainX, trainy, epochs=300, batch_size=8, verbose=1)
#model.fit(xtrain, ytrain, batch_size=256,epochs=25,  validation_split=.2, verbose=1)
acc = model.evaluate(trainX, trainy)
print("Loss:", acc[0], " Accuracy:", acc[1])
print("Training time --- %s seconds ---" % (time.time() - start_time))

1197 6 12


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)               │ (None, 1196, 16)       │           208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 598, 16)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 597, 32)        │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ (None, 298, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 297, 64)        │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ (None, 148, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 9472)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │     2,425,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 12)             │           780 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,472,444 (9.43 MB)

 Trainable params: 2,472,444 (9.43 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.0941 - loss: 2.6759
Epoch 2/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4857 - loss: 1.5483
Epoch 3/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7458 - loss: 0.7369
Epoch 4/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9553 - loss: 0.1534
Epoch 5/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.0284
Epoch 6/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9809 - loss: 0.0878
Epoch 7/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9848 - loss: 0.0783
Epoch 8/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.0200
Epoch 9/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.0045
Epoch 10/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.0020
Epoch 11/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.0016
Epoch 12/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

In [47]:
model.evaluate(testX, testy, verbose=1)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8698 - loss: 0.5555


[0.7338306903839111, 0.8333333134651184]

In [48]:
#Computing F1-score
import numpy as np
import sklearn
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, roc_auc_score
train_features = np.array(trainX)
test_features = np.array(testX)
train_labels=np.array(trainy)
#train_labels=pd.DataFrame(trainy)
#n_values = train_labels.idxmax(axis=1)
y_values=np.argmax(train_labels,axis=1)
#print(y_values)
test_labels=np.array(testy)
yt_values=np.argmax(test_labels,axis=1)
#train_predictions_baseline = model.predict_classes(train_features, batch_size=150)
a=model.predict([test_features])
predict_classes=np.argmax(a,axis=1)
th=.8
atempo=a.max(axis=1)
print(atempo.shape)
print('Rate of Probability of classification exceeding a threshold in test',(atempo[atempo>th].shape[0])/atempo.shape[0])
#prob2=pd.DataFrame(model.predict_proba(test_features,batch_size=150))
#a=prob2.max(axis=1)
#print('Probability of classification',(a[a>.80].shape[0])/prob2.shape[0])
#print('Rate of Probability of classification exceding a trhershold',(a[a>.50].shape[0])/a.shape[0])
a1=model.predict([train_features])
predict_classes1=np.argmax(a1,axis=1)
#f1_train=sklearn.metrics.f1_score(ytrain, train_predictions_baseline, average="weighted")
#test_predictions_baseline = model.predict_classes(test_features, batch_size=150)
#print(test_predictions_baseline)
#f1_test=sklearn.metrics.f1_score(test_labels, test_predictions_baseline, average="weighted")
#print('f1_scores in testing set',f1_test)
#Calculating metrics for each class
print("EVALUATION ON TESTING DATA")
print(classification_report(yt_values, predict_classes))

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step 

C:\Users\User\anaconda3\Lib\site-packages\keras\src\models\functional.py:238: UserWarning:

The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor_67
Received: inputs=('Tensor(shape=(32, 1197, 6))',)

C:\Users\User\anaconda3\Lib\site-packages\keras\src\models\functional.py:238: UserWarning:

The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor_67
Received: inputs=('Tensor(shape=(None, 1197, 6))',)



3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
(72,)
Rate of Probability of classification exceeding a threshold in test 0.9027777777777778
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
EVALUATION ON TESTING DATA
              precision    recall  f1-score   support

           0       0.75      1.00      0.86         6
           1       0.67      1.00      0.80         6
           2       1.00      0.33      0.50         6
           3       1.00      0.83      0.91         6
           4       1.00      1.00      1.00         6
           5       0.71      0.83      0.77         6
           6       1.00      1.00      1.00         6
           7       0.75      1.00      0.86         6
           8       0.71      0.83      0.77         6
           9       1.00      0.67      0.80         6
          10       1.00      0.83      0.91         6
          11       0.80      0.67      0.73         6

    accuracy                           0.83        72
   macro avg       0.87      0.83      0.82   

In [49]:
cm=confusion_matrix(yt_values, predict_classes)
print(cm)
good=np.diag(cm)/np.unique(yt_values,return_counts=True)[1]
print(good)

[[6 0 0 0 0 0 0 0 0 0 0 0]
 [0 6 0 0 0 0 0 0 0 0 0 0]
 [1 3 2 0 0 0 0 0 0 0 0 0]
 [0 0 0 5 0 0 0 1 0 0 0 0]
 [0 0 0 0 6 0 0 0 0 0 0 0]
 [0 0 0 0 0 5 0 0 1 0 0 0]
 [0 0 0 0 0 0 6 0 0 0 0 0]
 [0 0 0 0 0 0 0 6 0 0 0 0]
 [0 0 0 0 0 0 0 0 5 0 0 1]
 [1 0 0 0 0 0 0 1 0 4 0 0]
 [0 0 0 0 0 0 0 0 1 0 5 0]
 [0 0 0 0 0 2 0 0 0 0 0 4]]
[1.         1.         0.33333333 0.83333333 1.         0.83333333
 1.         1.         0.83333333 0.66666667 0.83333333 0.66666667]


In [50]:
import plotly.figure_factory as ff
# invert z idx values

z= cm

# invert z idx values
#z = z[::-1]

x= [0,1,2,3,4,5,6,7,8,9,10,11]
y1=x
#y =x[::-1].copy() # invert idx values of x
#print(y)
#y=x
# change each element of z to type string for annotations
z_text = [[str(y1) for y1 in x] for x in z]

# set up figure
fig = ff.create_annotated_heatmap(z, x=x, y=y1, annotation_text=z_text, colorscale='Viridis')

# add title
fig.update_layout(title_text='<i><b>Confusion matrix</b></i>',
                  #xaxis = dict(title='x'),
                  #yaxis = dict(title='x')
                 )

# add custom xaxis title
fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=0.5,
                        y=-0.15,
                        showarrow=False,
                        text="Predicted value",
                        xref="paper",
                        yref="paper"))

# add custom yaxis title
fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=-0.35,
                        y=0.5,
                        showarrow=False,
                        text="Actual value",
                        textangle=-90,
                        xref="paper",
                        yref="paper"))

# adjust margins to make room for yaxis title
fig.update_layout(margin=dict(t=50, l=200))

# add colorbar
fig['data'][0]['showscale'] = True
fig.show()

In [51]:
results1=model.evaluate(Xtb, yt, verbose=1)
print(results1)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7409 - loss: 1.2218
[1.4756274223327637, 0.7083333134651184]
